# CE311K Final Project Submission

## *Abstract*
In this project, we seek to create a VRP solver. The VRP solver will be generic in nature, and will take in a generalized spreadsheet. As a result, the only constraint on what can and cannot be solved by our code will be whether or not the address is registered with Google and TomTom, the two API's utilized in this project. 

## *Project Steps*
* Prep Data by adding coordinates
* Describe constraints as numpy arrays
* Write contraints to LP file format
* Solve LP File
* Interpret solution
* Graph solution

### *Libraries Used*

In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import json
from bs4 import BeautifulSoup as bs
import plotly as plt
import plotly.graph_objects as go
import shutil

## *Prepping Data*
The data intake will be in a fairly generic form. There are 2 columns: `Name` and `Address`. For later graphing, we will also need the coordinates of all our locations. Therefore, let us write a quick function to prepare the data.

In [ ]:
FILENAME = 'Whatever File You Wish To Use' # This is the name of the .csv file you wish to use. DO NOT INCLUDE .csv when specifying

In the event we receive multiple results from the API, let us create a handler. Since this is used later by the API caller, we need to define the function now.

In [ ]:
# Please note that this was all written in several scripts, and then dumped into a jupyer notebook.
# For best results, please ask Stephen Smith (UT EID: sjs5555, email: stephen.smith@utexas.edu) for a copy of his API keys and git repo
def MultipleResultsHandler(response:dict) -> int:
    ''' Has the user select which response they want from the API call when there are several

    # Arguments #
    :arg response: All of the possible responses from the API
    :type response: dict

    # Returns #
    :ret selection: The choice of the user
    :rtype selection: int
    '''
    location = response["summary"]["query"]
    print(f"There were multiple results querying for {location}, please select an option.\n------------------------------------------------------------------------------------")
    for index,item in enumerate(response["results"]):
        item_address = item["address"]["freeformAddress"]
        print(f"{index}) {item_address}")
        
    print("------------------------------------------------------------------------------------")
    selection = -1
    while (selection not in range(response["summary"]["numResults"])):
        selection = input("Please select an option from the ones given above: ")
        try:
            selection = int(selection)
        except ValueError:
            print("Please enter a valid digit from the given options.")
    return int(selection)

Now that the exception has been handled, let us write the API call function:

In [ ]:
def PopulateDataframe(df:pd.DataFrame,apiKey:str,pickFirst:bool=False) -> pd.DataFrame:
    ''' Takes a dataframe of addresses, and gives it coordinates

    # Arguments #
    :arg df: Loaded .csv file with all of the locations wanted
    :type df: pd.DataFrame
    :arg apiKey: TomTom api key
    :type apiKey: str
    :arg pickFirst: Option for user to just always pick the first option, instead of manually picking each option. Defaults to False
    :type pickFirst: bool

    # Returns #
    :ret df: Updated dataframe with all of the associated coordinates
    :rtype df pd.DataFrame
    '''
    assert 'Address' in df.columns, 'Ensure that the Dataframe provided contains a column called "Name".'
    df["Latitude"] = np.nan
    df["Longitude"] = np.nan
    for i in range(len(df)):
        location = df.iloc[i]["Address"]
        if pickFirst:
            response = requests.get(f"https://api.tomtom.com/search/2/geocode/{location}.json?limit=1&key={apiKey}").json()
        else:
            response = requests.get(f"https://api.tomtom.com/search/2/geocode/{location}.json?key={apiKey}").json()
        assert response["summary"]["numResults"] > 0, f"There were no results returned after querying for {location}, are you sure that's a valid location?"
        if (response["summary"]["numResults"] > 1):
            selection = MultipleResultsHandler(response)
            df.at[i,"Latitude"] = response["results"][selection]["position"]["lat"]
            df.at[i,"Longitude"] = response["results"][selection]["position"]["lon"]
        else:
            df.at[i,"Latitude"] = response["results"][0]["position"]["lat"]
            df.at[i,"Longitude"] = response["results"][0]["position"]["lon"]
    # Overwrite old file and return DataFrame
    path = os.path.join(os.getcwd(),'Data',FILENAME+'.csv')
    if os.path.exists(path):
        os.remove(path)
    df.to_csv(path, index='False')
    return df

Now that we have the function to prep the data, let us go ahead and do it!

In [ ]:
# Load needed data, apiKey
df = pd.read_csv(os.path.join(os.getcwd(),'Data',FILENAME+'.csv'))
with open(os.path.join(os.getcwd(),'Keys.json'), 'r') as f:
    keys = json.load(f)
# Update the dataframe
coords = PopulateDataframe(df, keys['TomTom'])
print(coords)

## *Modeling the Data*
VRP modeling is relatively straight forward. For reference, conduct a literature review. A helpful video might be [this one](https://www.youtube.com/watch?v=-hGL39jdtQE&ab_channel=Hern%C3%A1nC%C3%A1ceres). We will use a cost matrix and 3 constraints:
* All locations must be visited exactly once
* All locations must have exactly one entering and one exiting vehicle
* Subtours must be strictly prohibited

For subtour elimination, we will use the Miller, Tucker, Zemlin Approach, [as described here.](https://faculty.math.illinois.edu/~mlavrov/slides/482-spring-2020/slides35.pdf)

### First, let us create a distance matrix
This will be done by creating iterating through the dataset previously made, and entering the values into a numpy array.

In [ ]:
# Define API Call Function
def RouteCaller(loc1:str, loc2:str,  key:str=None, units:str='meters', routeType:str='BICYCLE') -> float:
    """ Function that makes a single API call between two distances

    # Arguments #
    :arg loc1: Starting location for the route
    :type loc1: str
    :arg loc2: Ending location for the route
    :type loc2: str
    :arg units: Measuring system for the route, metric of imperial
    :type units: str
    :arg routeType: Mode of travel for the route. Default is Bicyle because that's the purpose of this project
    :type routeType: str
    """
    print(f'Getting data for {loc1} to {loc2}')
    # Verify Key exists
    if key == None:
        raise ValueError('No API Key has been passed. Please insert key and try again')
    # Ping the API
    retVal = requests.get(
        f"https://maps.googleapis.com/maps/api/directions/json?destination={loc1}&origin={loc2}&units={units}&mode={routeType}&key={key}").json()
    if 'error_message' in retVal.keys():
        # Error handling
        print(
            f"Error when pinging API, issue: {retVal['error_message']}")
        return np.NaN
    else:
        # Return the route information
        return float(retVal['routes'][0]['legs'][0]['distance']['text'][:-3])

# Define the Distance Matrix
def generateDistanceMatrix(df:pd.DataFrame, key:str) -> np.ndarray:
    """ Takes in a DataFrame, and returns a numpy array

    # Arguments #
    :arg df: all of the addresses in the problem, placed in a pandas DataFrame
    :type df: pd.DataFrame
    :arg key: API key, so that Google knows who is calling
    :type key: str

    # Returns #
    :return distMatrix: A matrix of size NxN, where N is the number of locations in DataFrame df
    :rtype distMatrix: np.ndarray
    """
    # Local array storage
    distDict = {}
    arrays = []
    i = 0
    # Iterate through the DataFrame
    for index1, row1 in df.iterrows():
        distDict[index1] = []
        for index2, row2 in df.iterrows():
            if index1 == index2:
                # If location equals itself then give 0 for distance, no need to ping the API
                distDict[index1].append(0)

            elif index1 != index2:
                i += 1
                # Ping Google's Destination API for route data
                distDict[index1].append(RouteCaller(df.iloc[index1]['Address'], df.iloc[index2]['Address'], key))
    print(f'Successfully made {i} calls to the Google Maps Route API')
    # Process data into arrays, stack, and return
    for key in distDict.keys():
        arrays.append(np.array(distDict[key]))
    distMatrix = np.vstack(arrays)
    return distMatrix


def validateCache(filename:str,df:pd.DataFrame,key:str=None) -> np.ndarray:
    """ Validate and load cached data, to prevent unnecessary API calls

    # Arguments #
    :arg filename: ending of filename to check if it exits. If it doesn't we will end up making it
    :type filename: str
    :arg df: If no cache, need data to pass to route generator
    :type df: pd.DataFrame
    :arg key: If no cache, need key for API call
    :type key: str

    # Returns #
    :return distMatrix: Returns a distmatrix
    :rtype distMatrix: np.ndarray
    """
    path = os.getcwd()
    # Define where the cache should be
    cache_filename = os.path.splitext(filename)[0] + ".npy"
    # Check for if the parent folder exists, if not make one
    if not os.path.exists(os.path.join(os.getcwd(),"CachedDistances")):
        os.mkdir(os.path.join(path,"CachedDistances"))
    # Check if the distance matrix doesn't exist, if so make one
    if not os.path.exists(os.path.join(path,"CachedDistances",cache_filename)):
        distMatrix = generateDistanceMatrix(df, key)
        np.save(os.path.join(path,"CachedDistances",cache_filename),distMatrix)
        return distMatrix
    # Since the matrix exists, just load it
    else:
        distMatrix = np.load(os.path.join(path, "CachedDistances", cache_filename))
        return distMatrix

Aside: If it is know that routes are symmetric, an optimization could be made by only querying the API if i < j, else set the value to the previously recorded value. However, since one way roads do exist and could be used, it is necessary to assume the VRP is asymetric. 

### Second, let us define constraints off of the objective

In [ ]:
def generateContraintMatrix(distMatrix:np.ndarray) -> np.ndarray:
    """ Generate the constraint matrix from given distance matrix

    # Arguments #
    :arg distMatrix: Big distance matrix that we calculated in the last step
    :type distMatrix: np.ndarray

    # Returns #
    :return retVal: Combined constraint matrix, with top and bottom halves stacked nicely
    :rtype retVal: np.ndarray
    """
    NumElements = len(distMatrix)
    """
        Generates the top portion of the constraint matrix, a 1 x N matrix of ones for index i in N. Here is an example with N = 3.
        [ 0 0 0 0 0 0 0 0 0 ]        [ 1 1 1 0 0 0 0 0 0 ]
        | 0 0 0 0 0 0 0 0 0 | --->   | 0 0 0 1 1 1 0 0 0 |
        [ 0 0 0 0 0 0 0 0 0 ]        [ 0 0 0 0 0 0 1 1 1 ]
    """
    retArrayTop = np.zeros(shape=(NumElements,NumElements**2))
    for index in range(NumElements):
        retArrayTop[index,index*NumElements:(index+1)*NumElements] = np.ones(shape=(1,NumElements))
    """
        Generates the bottom portion of the constraint matrix, a N x N**2 matrix full of horizontally aligned identity N x N matricies.
        Example with N = 3.
        [ 1 0 0 ]   [ 1 0 0 ]   [ 1 0 0 ]         [ 1 0 0 1 0 0 1 0 0 ]
        | 0 1 0 | + | 0 1 0 | + | 0 1 0 |   --->  | 0 1 0 0 1 0 0 1 0 |
        [ 0 0 1 ]   [ 0 0 1 ]   [ 0 0 1 ]         [ 0 0 1 0 0 1 0 0 1 ]
    """
    individualBottoms = tuple(np.identity(NumElements) for i in range(NumElements))
    retArrayBottom = np.hstack(individualBottoms)
    retVal = np.vstack((retArrayTop,retArrayBottom))
    #retVal = np.vstack((retVal,np.ones(shape=(1,NumElements**2),dtype=np.uint8)))
    return retVal

It should be noted that this does not generate sub-tour elimination constraints. They are defined during the LP generation, as they utilize a different variable set.

### Writing the LP File
LP files are wonderful for Linear Programming, and will allow us to seamlessly use [NEOS](https://neos-server.org/neos/) as our free solver. Once the LP file is generated, it is a simple process to upload to a headless CPLEX solver and get an answer.

In [ ]:
def lpGenerator(distMatrix:np.ndarray, constraintMatrix:np.ndarray,filename:str) -> None:
    """ Writes the lp file for the constraint matrix. LP Files are the way we give the solver our problem.

    # Arguments #
    :arg distMatrix: a distance matrix of size N x N. This contains the necessary objective information
    :type distMatrix: np.ndarray
    :arg constraintMatrix: the constraint matrix with binary variables
    :type constraintMatrix: np.ndarray
    :arg filename: input filename that we will make the associated lp file for
    :type filename: str
    """
    path = os.getcwd()
    # Define where the LP file should be
    lp_filename = os.path.splitext(filename)[0] + ".lp"
    # Check for if the parent folder exists, if not make one
    if not os.path.exists(os.path.join(path,"LPFiles")):
        os.mkdir(os.path.join(path,"LPFiles"))
    full_lp_filename = os.path.join(path,"LPFiles",lp_filename)
    # Delete old lp file of same name
    if os.path.exists(full_lp_filename):
        os.remove(full_lp_filename)

    # Find N
    NumElements = len(distMatrix)

    # Variable Creation
    subtour_vars = {}
    vars_dict = {}
    for i in range(NumElements):
        subtour_vars[i] = f't{i}'
        vars_dict[i] = {}
        for j in range(NumElements):
            vars_dict[i][j] = f'i{i}j{j}'
    subtour_total = 0
    for i in range(NumElements):
        subtour_total += i
    # -------------- Objective Writing -------------- #
    costs = []
    for i in vars_dict.keys():
        for j in vars_dict[i].keys():
            cost = distMatrix[i, j]
            if cost != 0.0:
                costs.append(f'{cost} {vars_dict[i][j]}')
                costs.append('+')
    costs.pop()
    # -------------- Constraint Writing -------------- #
    lp_constraints = []
    # Top Third (One route in)
    loc = 0
    for i in vars_dict.keys():
        local = []
        for j in vars_dict[i].keys():
            if i != j:
                if constraintMatrix[i,j+loc] == 1.0:
                    local.append(vars_dict[i][j])
                    local.append(' + ')
        local.pop()
        lp_constraints.append(local)
        loc += NumElements
    # Second Third (One route out)
    loc = 0
    for i in vars_dict.keys():
        local = []
        for j in vars_dict[i].keys():
            if i != j:
                if constraintMatrix[i+NumElements,j*NumElements+loc] == 1.0:
                    local.append(vars_dict[j][i])
                    local.append(' + ')
        local.pop()
        lp_constraints.append(local)
        loc += 1
    # Bottom Third (Subtour elimination)
    subtours =[]
    for i in vars_dict.keys():
        for j in vars_dict[i].keys():
            if i != j:
                if i == 0:
                    # Set i == 0 as the first location visited
                    subtours.append(f'{vars_dict[i][j]} = 1 -> {subtour_vars[i]} = 1')
                else:
                    # Rest of the subtour elimination clause
                    subtours.append(f'{vars_dict[i][j]} = 1 -> {subtour_vars[i]} - {subtour_vars[j]} >= 1')
    subtour_final = []
    for i in subtour_vars.keys():
        subtour_final.append(i)
        subtour_final.append(' + ')
    subtour_final.pop()
    subtour_final.append(f'= {subtour_total}')

    # Write the LP file
    with open(full_lp_filename, 'x') as lp:
        # Objective Section
        lp.write('Min \n')
        for eq in costs:
            lp.write(f'{eq} ')
        # Constraint Section
        lp.write('\nsubject to \n')
        for eq in lp_constraints:
            for i in eq:
                lp.write(i)
            lp.write(' = 1 \n')        
        count = 0
        for i in subtours:
            lp.write(f'\nGC{count}: {i}')
            count += 1
        for i in subtour_final:
            lp.write(f'{i}')
        # Define Subtour Variable Bounds
        lp.write('\nbounds \n')
        for i in subtour_vars.keys():
            lp.write(f'0 <= {subtour_vars[i]} <= {NumElements} \n')
        # Define Variable Types
        lp.write('bin \n')
        for i in vars_dict.keys():
            for j in vars_dict[i].keys():
                if i != j:
                    lp.write(f'{vars_dict[i][j]} ')
        lp.write('\nint \n')
        for i in subtour_vars.keys():
            lp.write(f'{subtour_vars[i]} ')
        lp.write('\nEND')
    # This is messy but it works. I am also unaware of a better method of writing LP files in Python.
    # Pulp might be an alternative, but I am unfamiliar with how to use it.

## *Interpreting the Solution*
Now that the LP file has been solved, NEOS has produced a .sol file in xml format that needs interpreting. For easy loading, BeautifulSoup4 is the xml reader of choice. Let us load and interpret the solution:

In [ ]:
# Variables used later
MAP_PATH = MAP_PATH = os.path.join(os.getcwd(), "Maps")
with open(os.path.join(os.getcwd(), 'Keys.json')) as f:
    keys = json.load(f)
TomTomKey = keys['TomTom']
MapBoxKey = keys['MapBox']

# Get the solution into a workable format
def solParser(distMatrix: np.ndarray, filename: str) -> dict:
    """ Read the .sol file and return a much easier to work with dictionary over this xml jargon

    # Arguments #
    :arg distMatrix: 
    :arg filename: Generic name of the input data file
    :type filename: str

    # Returns #
    :ret solution: Usable solution in dictionary form
    :rtype solution: dict
    """
    # Load solution file
    with open(os.path.join(os.getcwd(), 'sol', filename+'.sol'), 'r') as f:
        data = f.read()
    BS_Data = bs(data, 'xml')
    # Get N from distMatrix
    NumElements = len(distMatrix)
    solution = {}
    for i in range(NumElements):
        for j in range(NumElements):
            val = BS_Data.find('variable', {'name': f'i{i}j{j}'})
            if val != None:
                solution[val.get('name')] = val.get('value')
    return solution

# Interpret the solution into something readable, print to terminal
def solInterpreter(sol: dict, distMatrix: np.ndarray, filename: str) -> dict:
    """ Takes a solution and outputs the route to the terminal

    # Arguments #
    :arg sol: All of the route variables and associated values for the solution
    :type sol: dict
    :arg distMatrix: Used to get how many locations there are
    :type distMatrix: np.ndarray
    :arg filename: Generic name of the input data file
    :type filename: str

    # Returns #
    :ret routes: The start- and end-points of each route
    :rtype routes: dict
    """
    # Get names of locations
    with open(os.path.join(os.getcwd(), 'Data', filename+'.csv'), "r") as data:
        df = pd.read_csv(data)
    NumElements = len(distMatrix)
    routes = {}
    print('# ---------- Interpreting Solution ---------- #')
    # Get the solution in order, ease of read
    for i in range(NumElements):
        for j in range(NumElements):
            loc = f'i{i}j{j}'
            if loc in sol.keys() and int(sol[loc]) == 1:
                routes[i] = j
    stop = 0
    i = 0

    # Print solution to terminal
    for loc in routes.keys():
        if stop != loc:
            print(
                f'Route contains {df.iloc[routes[stop], 1]} -> {df.iloc[routes[loc], 1]}')
            stop = loc
        elif i != 0:
            print(f'Possible circular route: {loc} -> {stop}, at index {i}')
        i += 1
    # Verify solution is legit
    if i == NumElements:
        print(
            f'Route contains {i} stops out of {NumElements} locations, and is indeed circular!')
    else:
        print(
            f'Route is not circular, as there are {i} stops and {NumElements} locations')
    print(routes)
    return routes

### This is good, but a graph is better
It is undeniable that a graph is much more succint and easier to read than a list of print statements, at least when looking at a route. Therefore, use Plotly Mapbox for ease of use/interpretation.

In [ ]:
def APIMANAGER(js: str) -> pd.DataFrame:
    ''' Gets all of the locations into a pretty format

    # Arguments #
    :arg js: return from the TomTom url that needs interpreting
    :type js: str
    
    # Returns #
    :ret loc_df: Full route coordinates for the route inside js
    :rtype loc_df: pd.DataFrame
    '''
    jsDump = json.dumps(js)
    jsLoad = json.loads(jsDump)
    points = jsLoad['routes'][0]['legs'][0]['points']
    coords = []
    for item in points:
        coords.append([float(item['longitude']), float(item['latitude'])])
    loc_df = pd.DataFrame(coords, columns=['Longitude', 'Latitude'])
    return loc_df

def routeGenerator(startLat: float, startLon: float, endLat: float, endLon: float, apiKey: str) -> pd.DataFrame:
    ''' Takes a start and end point, and produces a full route

    # Arguments #
    :arg startLat: Latitude for starting location
    :type startLat: float
    :arg startLon: Longitude for starting location
    :type startLon: float
    :arg endLat: Latitude for ending location
    :type endLat: float    
    :arg endLon: Longitude for ending location
    :type endLon: float
    :arg apiKey: TomTom api key
    :type apiKey: str

    # Returns #
    :ret df: All of the coordinates needed for the route
    :rtype df: pd.DataFrame
    '''
    tomtomURL = f'https://api.tomtom.com/routing/1/calculateRoute/{startLat},{startLon}:{endLat},{endLon}/json?maxAlternatives=0&routeType=shortest&travelMode=bicycle&key={apiKey}'
    getData = requests.get(tomtomURL)
    while (getData.status_code != 200):
        getData = requests.get(tomtomURL)
    jsonTomTomString = getData.json()
    start = pd.DataFrame({'Latitude':startLat,'Longitude':startLon}, index =[0])
    df = APIMANAGER(jsonTomTomString)
    end = pd.DataFrame({'Latitude':endLat,'Longitude':endLon}, index =[0])
    df = pd.concat([start,df[:],end]).reset_index(drop = True)
    return df


def GenerateMapSolutions(sol: dict, dataframe: pd.DataFrame, apiKey: str) -> dict:
    """Creates a pandas dataframe that represents a solution from the response given by a .sol file

    # Arguments #
    :arg sol: A full circular route
    :type response: dict
    :arg dataframe: A dataframe consisting of at least location Longitude and Latitudes.
    :type dataframe: pd.DataFrame
    :arg apiKey: apiKey for TomTom
    :type apiKey: str

    # Returns #
    :return: A dataframe which represents a series of points of different coordinates and colors that form a route.
    :rtype: pd.DataFrame
    """
    retVal = {}
    for i in sol.keys():
        startLat = dataframe.iloc[i]["Latitude"].item()
        startLon = dataframe.iloc[i]["Longitude"].item()
        endLat = dataframe.iloc[sol[i]]["Latitude"].item()
        endLon = dataframe.iloc[sol[i]]["Longitude"].item()
        print((startLat, startLon), "->", (endLat, endLon))
        retVal[i] = routeGenerator(startLat, startLon, endLat, endLon, apiKey)
    return retVal


def make_map(pathingList:dict, locations:pd.DataFrame, mapboxKey:str, sol:dict) -> None:
    ''' Now that all the prep is done, actually make the darn map

    # Arguments #
    :arg pathingList: Nested dataframes with all of the route coordinates
    :type pathingList: dict
    :arg locations: all of the locations that will be visited
    :type locations: pd.DataFrame
    :arg mapboxKey: key to access Plotly Mapbox
    :type mapboxKey: str
    :arg sol: Ordered solution, so that the routes can be labeled
    :type sol: dict
    '''
    fig = go.Figure(go.Scattergeo())
    # Plot all locations
    lat = locations['Latitude'].values.tolist()
    lon = locations['Longitude'].values.tolist()
    fig.add_trace(go.Scattermapbox(
        lat=lat,
        lon=lon,
        name='Locations'
    ))
    # Plot all routes
    for i in pathingList.keys():
        start = locations.iloc[i]['Name']
        end = locations.iloc[sol[i]]['Name']
        fig.add_trace(go.Scattermapbox(
            mode='lines',
            lat=pathingList[i]['Latitude'].values.tolist(),
            lon=pathingList[i]["Longitude"].values.tolist(),
            name = f'{start} -> {end}'
            )
        )

    # Using Mapbox
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=mapboxKey)
    # Display map
    plt.offline.plot(
        fig,
        filename=os.path.join(os.getcwd(), "Maps", f"GeneratedMap.html"),
        auto_open=True,
    )


def ShowMapSolutions(sol: dict, dataframe: pd.DataFrame, TomTomKey: str, MapBoxKey: str):
    """Generates map soutions from a .sol file

    :param sol: A route dictionary
    :type Solutions: dict
    :param dataframe: A dataframe consisting of at least location Longitude and Latitudes.
    :type dataframe: pd.DataFrame
    :param TomTomKey: API key for TomTom, the route provider
    :type TomTomKey: str
    :param MapBoxKey: API key for Plotly Mapbox, the route grapher
    :type MapBoxKey: str
    """
    if os.path.exists(MAP_PATH):
        shutil.rmtree(MAP_PATH)
    os.mkdir(MAP_PATH)
    GeneratedSolution = GenerateMapSolutions(sol, dataframe, TomTomKey)
    make_map(GeneratedSolution, dataframe, MapBoxKey, sol)

For example graphs, LP files, and data, please reference [this google drive folder](https://drive.google.com/drive/folders/1wWpx6V2ewvnvsE55xQsGaKLdma_f0OjR?usp=sharing).